# 混同行列の理解と実装

このノートブックでは、混同行列の基本概念から実装、可視化まで詳しく学習します。

## 学習目標
- 混同行列の基本概念と要素の理解
- 混同行列の手動実装
- 可視化と解釈
- 実務での活用方法


In [ ]:
# 必要なライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")


## 1. 混同行列の基本概念


In [ ]:
def confusion_matrix_manual(y_true, y_pred):
    """
    混同行列の手動実装
    
    Parameters:
    y_true: 実際のラベル
    y_pred: 予測ラベル
    
    Returns:
    cm: 混同行列
    """
    # クラスの取得
    classes = np.unique(np.concatenate([y_true, y_pred]))
    n_classes = len(classes)
    
    # 混同行列の初期化
    cm = np.zeros((n_classes, n_classes), dtype=int)
    
    # 各予測の結果をカウント
    for i in range(len(y_true)):
        true_class = y_true[i]
        pred_class = y_pred[i]
        
        # クラスインデックスの取得
        true_idx = np.where(classes == true_class)[0][0]
        pred_idx = np.where(classes == pred_class)[0][0]
        
        cm[true_idx, pred_idx] += 1
    
    return cm, classes

# 簡単な例で混同行列を理解
y_true_example = np.array([0, 1, 0, 1, 0, 1, 0, 1])
y_pred_example = np.array([0, 1, 1, 1, 0, 0, 0, 1])

cm_manual, classes = confusion_matrix_manual(y_true_example, y_pred_example)
cm_sklearn = confusion_matrix(y_true_example, y_pred_example)

print("=== 混同行列の基本例 ===")
print(f"実際のラベル: {y_true_example}")
print(f"予測ラベル: {y_pred_example}")
print(f"クラス: {classes}")
print(f"\n手動実装の混同行列:")
print(cm_manual)
print(f"\nscikit-learnの混同行列:")
print(cm_sklearn)
print(f"\n実装の一致: {np.array_equal(cm_manual, cm_sklearn)}")

# 混同行列の要素の説明
print("\n=== 混同行列の要素 ===")
print("2×2混同行列の場合:")
print("                予測")
print("            0       1")
print("実際  0    TN      FP")
print("     1    FN      TP")
print("\n要素の説明:")
print("TP (True Positive): 正しく正例と予測")
print("TN (True Negative): 正しく負例と予測")
print("FP (False Positive): 間違って正例と予測")
print("FN (False Negative): 間違って負例と予測")


## 2. 混同行列の可視化


In [ ]:
def plot_confusion_matrix(cm, classes, title='Confusion Matrix', cmap=plt.cm.Blues):
    """
    混同行列の可視化
    
    Parameters:
    cm: 混同行列
    classes: クラス名
    title: グラフのタイトル
    cmap: カラーマップ
    """
    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)
    
    # 数値の表示
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
    
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()

# 混同行列の可視化
plot_confusion_matrix(cm_manual, classes, 'Confusion Matrix Example')

# より詳細な可視化
def plot_detailed_confusion_matrix(cm, classes):
    """
    詳細な混同行列の可視化
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 基本的な混同行列
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax1,
                xticklabels=classes, yticklabels=classes)
    ax1.set_title('Confusion Matrix')
    ax1.set_ylabel('True Label')
    ax1.set_xlabel('Predicted Label')
    
    # 正規化された混同行列
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cm_normalized, annot=True, fmt='.2f', cmap='Blues', ax=ax2,
                xticklabels=classes, yticklabels=classes)
    ax2.set_title('Normalized Confusion Matrix')
    ax2.set_ylabel('True Label')
    ax2.set_xlabel('Predicted Label')
    
    plt.tight_layout()
    plt.show()

plot_detailed_confusion_matrix(cm_manual, classes)


## 3. 実データでの混同行列


In [ ]:
# 乳がんデータセットでの混同行列
cancer = load_breast_cancer()
X, y = cancer.data, cancer.target

# データの分割と標準化
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ロジスティック回帰モデルの訓練
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)

# 予測
y_pred = model.predict(X_test_scaled)

# 混同行列の計算
cm_cancer = confusion_matrix(y_test, y_pred)
classes_cancer = ['Malignant', 'Benign']

print("=== 乳がんデータセットでの混同行列 ===")
print(f"テストデータの形状: {X_test.shape}")
print(f"クラスの分布: {np.bincount(y_test)}")
print(f"混同行列:")
print(cm_cancer)

# 混同行列の可視化
plot_detailed_confusion_matrix(cm_cancer, classes_cancer)

# 混同行列から基本指標を計算
def calculate_metrics_from_cm(cm):
    """
    混同行列から基本指標を計算
    """
    if cm.shape == (2, 2):
        tn, fp, fn, tp = cm.ravel()
        
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        return {
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall,
            'Specificity': specificity,
            'F1-score': f1
        }
    else:
        return None

# 指標の計算
metrics = calculate_metrics_from_cm(cm_cancer)
if metrics:
    print("\n=== 混同行列から計算した指標 ===")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

# 分類レポートとの比較
print("\n=== scikit-learnの分類レポート ===")
print(classification_report(y_test, y_pred, target_names=classes_cancer))


## 4. 多クラス分類の混同行列


In [ ]:
# 多クラス分類用のデータセット
X_multi, y_multi = make_classification(
    n_samples=1000,
    n_features=4,
    n_redundant=0,
    n_informative=4,
    n_clusters_per_class=1,
    n_classes=3,
    random_state=42
)

# データの分割と標準化
X_train_multi, X_test_multi, y_train_multi, y_test_multi = train_test_split(
    X_multi, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

scaler_multi = StandardScaler()
X_train_multi_scaled = scaler_multi.fit_transform(X_train_multi)
X_test_multi_scaled = scaler_multi.transform(X_test_multi)

# 多クラス分類モデルの訓練
model_multi = LogisticRegression(multi_class='multinomial', random_state=42, max_iter=1000)
model_multi.fit(X_train_multi_scaled, y_train_multi)

# 予測
y_pred_multi = model_multi.predict(X_test_multi_scaled)

# 混同行列の計算
cm_multi = confusion_matrix(y_test_multi, y_pred_multi)
classes_multi = ['Class 0', 'Class 1', 'Class 2']

print("=== 多クラス分類の混同行列 ===")
print(f"テストデータの形状: {X_test_multi.shape}")
print(f"クラスの分布: {np.bincount(y_test_multi)}")
print(f"混同行列:")
print(cm_multi)

# 多クラス混同行列の可視化
plot_detailed_confusion_matrix(cm_multi, classes_multi)

# 多クラス分類の指標計算
def calculate_multiclass_metrics(cm):
    """
    多クラス分類の指標を計算
    """
    n_classes = cm.shape[0]
    
    # 各クラスの指標
    class_metrics = {}
    for i in range(n_classes):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        tn = cm.sum() - tp - fp - fn
        
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        class_metrics[f'Class {i}'] = {
            'Precision': precision,
            'Recall': recall,
            'F1-score': f1
        }
    
    # Macro平均
    macro_precision = np.mean([class_metrics[f'Class {i}']['Precision'] for i in range(n_classes)])
    macro_recall = np.mean([class_metrics[f'Class {i}']['Recall'] for i in range(n_classes)])
    macro_f1 = np.mean([class_metrics[f'Class {i}']['F1-score'] for i in range(n_classes)])
    
    # Micro平均（全体のAccuracy）
    micro_accuracy = np.trace(cm) / cm.sum()
    
    return class_metrics, macro_precision, macro_recall, macro_f1, micro_accuracy

# 指標の計算
class_metrics, macro_precision, macro_recall, macro_f1, micro_accuracy = calculate_multiclass_metrics(cm_multi)

print("\n=== 多クラス分類の指標 ===")
print("各クラスの指標:")
for class_name, metrics in class_metrics.items():
    print(f"\n{class_name}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")

print(f"\nMacro平均:")
print(f"  Precision: {macro_precision:.4f}")
print(f"  Recall: {macro_recall:.4f}")
print(f"  F1-score: {macro_f1:.4f}")

print(f"\nMicro平均 (Accuracy): {micro_accuracy:.4f}")

# scikit-learnの分類レポートとの比較
print("\n=== scikit-learnの分類レポート ===")
print(classification_report(y_test_multi, y_pred_multi, target_names=classes_multi))


## 5. 演習問題

### 演習1: 混同行列の手動実装
異なるデータセットで混同行列を手動実装し、scikit-learnの結果と比較してみましょう。

### 演習2: 可視化の改善
混同行列の可視化を改善し、より見やすいグラフを作成してみましょう。

### 演習3: クラス不均衡の影響
クラス不均衡データで混同行列を確認し、指標の変化を観察してみましょう。


## まとめ

このノートブックでは、混同行列の基本概念から実装、可視化まで詳しく学習しました。

**学習した内容**：
- 混同行列の基本概念と要素
- 手動実装とscikit-learnの比較
- 二値分類と多クラス分類での混同行列
- 混同行列から基本指標の計算
- 可視化と解釈

**重要なポイント**：
- 混同行列は分類性能の詳細な分析に重要
- 二値分類ではTP、TN、FP、FNの4つの要素
- 多クラス分類では各クラスの性能を個別に評価
- 可視化により直感的な理解が可能

**次のステップ**：
- 基本評価指標の詳細学習
- Precision-Recall曲線の理解
- ROC曲線とAUCの学習
